In [1]:
import sys
print(sys.executable)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1730775316289_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/bin/python3

In [3]:
import io
import numpy as np
import pandas as pd
from PIL import Image
from keras import Model
from keras.src.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.src.utils import img_to_array
from pyspark.ml.feature import PCA
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AWS CONFIG

In [4]:
DATA_PATH = "s3a://xavier-project-8-bucket/Test/"
RESULTS_PATH = "s3a://xavier-project-8-bucket/results/"
PCA_RESULTS_PATH = "s3a://xavier-project-8-bucket/results_pca/"

spark = (SparkSession
         .builder
         .appName('project-8-spark-application')
         .config("spark.executor.memory", "15g") # C5.xlarge instances has 16gb of memory
         .config("spark.driver.memory", "15g")
         .config("spark.executor.cores", "4")  # C5.xlarge instanced have 4 vcpus
         .config("spark.sql.parquet.writeLegacyFormat", 'true')
         .getOrCreate()
         )
sc = spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#MISC CONFIG

pd.options.display.max_columns = None
pd.options.display.max_rows = None
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [7]:
def load_model():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
    
    for layer in model.layers:
        layer.trainable = False
        
    new_model = Model(inputs=model.input, outputs=model.layers[-2].output)
    new_model.set_weights(broadcasted_weights.value)
    
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    
    return preprocess_input(arr)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    # array_to_vector = udf(lambda x: Vectors.dense(x), VectorUDT())
    # spark_df = features_df.withColumn("features", array_to_vector("features"))
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches. This amortizes the overhead of loading big models.
    model = load_model()
    
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1730775316289_0002/container_1730775316289_0002_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [11]:
print("Starting Spark script.\n")

images = spark.read.format("binaryFile") \
    .option("pathGlobFilter", "*.jpg") \
    .option("recursiveFileLookup", "true") \
    .load(DATA_PATH)

print(f"Loaded {images.count()} images from {DATA_PATH}.\n")

images = images.withColumn('label', element_at(split(images['path'], '/'), -2))

# Transfer learning, we don't keep the last layer
model = MobileNetV2(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)
broadcasted_weights = sc.broadcast(new_model.get_weights())
print("The weights have been broadcasted.\n")

# Featurize images and saving the results
numbers_to_float_udf = udf(lambda x: [float(number) for number in x], ArrayType(FloatType()))
array_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())
features_df = (images
               .repartition(20)
               .withColumn("features", featurize_udf("content"))
               .withColumn("features", numbers_to_float_udf("features"))
               .withColumn("features", array_to_vector_udf("features"))
               .select("path", "label", "features"))
features_df.write.mode("overwrite").parquet(RESULTS_PATH)
print(f"The features have been saved to {RESULTS_PATH}.\n")


# Load the results back
spark_df = spark.read.parquet(RESULTS_PATH)
print("The results have been loaded back.\n")

# Missing PCA step added
pca = PCA(k=42, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(spark_df)
df_pca = pca_model.transform(spark_df)

# Saving df with PCA as parquet and csv
(df_pca.write.mode("overwrite")
 .parquet(PCA_RESULTS_PATH))
(df_pca
 .withColumn("features", col("features").cast("string"))
 .withColumn("pca_features", col("pca_features").cast("string"))
 .write
 .mode("overwrite")
 .option("header", "true")
 .format("csv")
 .save(f"{PCA_RESULTS_PATH}_csv"))

explained_variance = pca_model.explainedVariance
# print("Explained variance ratio:", explained_variance)
print(f"Total variance: {sum(explained_variance.toArray())}.\n")


# Load the results back again
spark_pca_df = spark.read.parquet(PCA_RESULTS_PATH)
print("The results with PCA have been loaded back.\n")

print(spark_pca_df.show(5))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Starting Spark script.

Loaded 22688 images from s3a://xavier-project-8-bucket/Test/
14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
The weights have been broadcasted.

The features have been saved to s3a://xavier-project-8-bucket/results/.

The results have been loaded back.

Explained variance ratio: [0.10140993376526122,0.08005980580315701,0.06350303180267557,0.05014785225124536,0.03535428254127414,0.029159929579133458,0.027732147239431282,0.02284890389166808,0.019859433649752786,0.019091521207050523]
The results with PCA have been loaded back.

+--------------------+--------------+--------------------+--------------------+
|                path|         label|            features|        pca_features|
+--------------------+--------------+--------------------+--------------------+
|s3a://xavier-proj...|    Watermelon|[0.13241590559482...|[-2.9173956108569...|
|s3a://xavier-proj...|Pineapple Mini|[0.0,5.1500864028...|[-4.7617026320549...|
|s3a://xavier-proj...|Pineapple Mini|[0.0,